### En este Notebook se realizaran limpieza de datos.
* Eliminar valores nulos en columnas importantes.


In [3]:
#exportar librerias necesarias
%matplotlib inline
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
#import ace_tools as tools
import os

In [33]:
warnings.filterwarnings("ignore")

### Importanción de dataset 

In [34]:
df_calendar = pd.read_csv('data/calendar.csv')
df_list_de = pd.read_csv('data/listings_detailed.csv')
df_list = pd.read_csv('data/listings.csv')
df_neighbour = pd.read_csv('data/neighbourhoods.csv')
df_reviws_de = pd.read_csv('data/reviews_detailed.csv')
df_reviws_su = pd.read_csv('data/reviews_summary.csv')

### Eliminar valores nulos 
1. Eliminar columnas con grandes cantidades de valores NaN en una columna

In [35]:
# Lista de datasets
datasets = [df_calendar, df_list_de, df_list, df_neighbour, df_reviws_de, df_reviws_su]

# Umbral para eliminar columnas, para este proyecto definimos 50% como unbral para eliminar columnas. 
umbral = 0.5

# Función para eliminar columnas con muchos valores NaN más del 50%
def eliminar_columnas_con_valores_nulos(df, umbral):
    # Calcular el porcentaje de valores nulos en cada columna
    null_percentage = df.isnull().mean()
    # Eliminar columnas que exceden el umbral de valores nulos
    df.drop(columns=null_percentage[null_percentage > umbral].index, inplace=True)

for df in datasets:
    eliminar_columnas_con_valores_nulos(df, umbral)

In [36]:
#verificamos si efectivamente se elimino la columna 
df_list.head()

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,6400,The Studio Milan,13822,Francesca,TIBALDI,45.44119,9.17813,Private room,100.0,4,10,2019-04-13,0.06,1,358,0
1,23986,""" Characteristic Milanese flat""",95941,Jeremy,NAVIGLI,45.44806,9.17373,Entire home/apt,180.0,1,27,2024-04-20,0.18,1,343,2
2,40470,Giacinto Cosy & clean flat near MM1,174203,Giacinto,VIALE MONZA,45.52023,9.22747,Entire home/apt,95.0,3,42,2024-02-29,0.26,2,353,2
3,46536,Nico & Cinzia's Pink Suite!,138683,Nico&Cinzia,VIALE MONZA,45.52276,9.22478,Entire home/apt,110.0,3,36,2024-05-12,0.24,1,3,6
4,59226,Near Piazza Gae Aulenti silent e reserved flat,244087,Francesca,CENTRALE,45.48201,9.19809,Entire home/apt,180.0,3,9,2022-06-13,0.06,1,177,0


In [37]:
# Imputar valores nulos en la columna 'price' con la mediana
df_list['price'].fillna(df_list['price'].median(), inplace=True)

df_list.to_csv('data_clean/listings.csv', index=False)

In [38]:
# Imputar valores nulos en 'last_review' utilizando el último valor registrado antes del nulo
# Convertir 'last_review' a formato de fecha para trabajar con la imputación
df_list['last_review'] = pd.to_datetime(df_list['last_review'], errors='coerce')

df_list['last_review'].fillna(method='ffill', inplace=True)

df_list.to_csv('data_clean/listings.csv', index=False)

In [39]:
# Imputar valores nulos en la columna 'reviews_per_month' con el valor 0
df_list['reviews_per_month'].fillna(0, inplace=True)

df_list.to_csv('data_clean/listings.csv', index=False)

In [40]:
print("Cantidad de valores nulos por columna:")
print(df_list.isnull().sum())

Cantidad de valores nulos por columna:
id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
number_of_reviews_ltm             0
dtype: int64


### Imputación en el dataset 'listings_detailed.csv'

In [41]:
# Identificar columnas con valores nulos y calcular el porcentaje de nulos
null_counts = df_list_de.isnull().sum()
null_percentage = (null_counts / len(df_list_de)) * 100
null_columns = null_counts[null_counts > 0].index

# Crear un resumen de las columnas con valores nulos
null_summary = pd.DataFrame({
    'Columna': null_columns,
    'Cantidad de Nulos': null_counts[null_columns],
    'Porcentaje de Nulos': null_percentage[null_columns]
})

null_summary

,Columna,Cantidad de Nulos,Porcentaje de Nulos
description,description,655,2.690380
neighborhood_overview,neighborhood_overview,11301,46.418303
host_location,host_location,4561,18.734084
host_about,host_about,11716,48.122895
host_response_time,host_response_time,3384,13.899614
host_response_rate,host_response_rate,3384,13.899614
host_acceptance_rate,host_acceptance_rate,1925,7.906843
host_is_superhost,host_is_superhost,808,3.318820
neighbourhood,neighbourhood,11301,46.418303
bathrooms,bathrooms,1238,5.085024


In [72]:
# imputar columnas del tipo 'texto' con el valor 'sin informacion'
columns_to_impute = ['description', 'neighborhood_overview', 'host_location', 'host_about']
impute_value = "Sin informacion"

df_list_de[columns_to_impute] = df_list_de[columns_to_impute].fillna(impute_value)

df_list_de.to_csv('data_clean/listings_detailed.csv', index=False)

In [73]:
# Imputar valores nulos en las columnas 'host_response_time' y 'host_is_superhost' con la moda de cada columna
for column in ['host_response_time', 'host_is_superhost']:
    mode_value = df_list_de[column].mode()[0]  # Calcular la moda
    df_list_de[column].fillna(mode_value, inplace=True)  # Imputar valores nulos con la moda

df_list_de.to_csv('data_clean/listings_detailed.csv', index=False)

In [75]:
# Imputar la columna 'host_response_rate' con la mediana del porcentaje de respuesta
# Convertir a numérico eliminando el símbolo '%' y luego imputar
#df_list_de['host_response_rate'] = pd.to_numeric(df_list_de['host_response_rate'].str.replace('%', ''), errors='coerce')
#df_list_de['host_response_rate'].fillna(df_list_de['host_response_rate'].median(), inplace=True)

# Imputar la columna 'host_acceptance_rate' con la mediana del porcentaje de aceptación
#df_list_de['host_acceptance_rate'] = pd.to_numeric(df_list_de['host_acceptance_rate'].str.replace('%', ''), errors='coerce')
#df_list_de['host_acceptance_rate'].fillna(df_list_de['host_acceptance_rate'].median(), inplace=True)

# Imputar 'bathrooms' con la mediana por tipo de propiedad (assumed 'property_type' exists in the dataset)
df_list_de['bathrooms'] = df_list_de.groupby('property_type')['bathrooms'].transform(lambda x: x.fillna(x.median()))

# Imputar 'bathrooms_text' usando la mediana de 'bathrooms'
median_bathrooms = df_list_de['bathrooms'].median()
df_list_de['bathrooms_text'] = df_list_de['bathrooms_text'].fillna(median_bathrooms)

# Imputar 'beds' con la mediana por tipo de propiedad
df_list_de['beds'] = df_list_de.groupby('property_type')['beds'].transform(lambda x: x.fillna(x.median()))

# Imputar las columnas 'review_scores_*' con la mediana de cada una de ellas
review_score_columns = [col for col in df_list_de.columns if col.startswith('review_scores_')]
for col in review_score_columns:
    df_list_de[col].fillna(df_list_de[col].median(), inplace=True)

# Guardar el DataFrame resultante en la misma ubicación, reemplazando el archivo original
df_list_de.to_csv('data_clean/listings_detailed.csv', index=False)

In [76]:
# Imputar valores nulos de las columnas con las técnicas especificadas
df_list_de['neighbourhood'].fillna("Desconocido", inplace=True)
df_list_de['has_availability'].fillna(df_list_de['has_availability'].mode()[0], inplace=True)
df_list_de['first_review'].fillna(method='ffill', inplace=True)
df_list_de['last_review'].fillna(method='ffill', inplace=True)

# Guardar el DataFrame resultante en la misma ubicación, reemplazando el archivo anterior
df_list_de.to_csv('data_clean/listings_detailed.csv', index=False)

In [77]:
# Convertir la columna 'price' a tipo numérico, eliminando caracteres no numéricos (como '$' y comas)
df_list_de['price'] = pd.to_numeric(df_list_de['price'].replace('[\$,]', '', regex=True), errors='coerce')

# Imputar los valores nulos en 'price' con la mediana de la columna
df_list_de['price'].fillna(df_list_de['price'].median(), inplace=True)

# Guardar el DataFrame resultante en la misma ubicación, reemplazando el archivo anterior
df_list_de.to_csv('data_clean/listings_detailed.csv', index=False)

In [78]:
# Imputar valores nulos en la columna 'reviews_per_month' con el valor 0
df_list_de['reviews_per_month'].fillna(0, inplace=True)

# Guardar el DataFrame resultante en la misma ubicación, reemplazando el archivo anterior
df_list_de.to_csv('data_clean/listings_detailed.csv', index=False)

In [80]:
print("Cantidad de valores nulos por columna:")
print(df_list_de.isnull().sum())

Cantidad de valores nulos por columna:
id                                              0
listing_url                                     0
scrape_id                                       0
last_scraped                                    0
source                                          0
                                               ..
calculated_host_listings_count                  0
calculated_host_listings_count_entire_homes     0
calculated_host_listings_count_private_rooms    0
calculated_host_listings_count_shared_rooms     0
reviews_per_month                               0
Length: 71, dtype: int64


In [4]:
#importamos los dataset limpio para verificar su estado de valores NaN
df_calendar = pd.read_csv('data_clean/calendar.csv')
df_list_de = pd.read_csv('data_clean/listings_detailed.csv')
df_list = pd.read_csv('data_clean/listings.csv')
df_neighbour = pd.read_csv('data_clean/neighbourhoods.csv')
df_reviws_de = pd.read_csv('data_clean/reviews_detailed.csv')
df_reviws_su = pd.read_csv('data_clean/reviews_summary.csv')

In [68]:
print("Cantidad de valores nulos por columna:")
print(df_reviws_su.isnull().sum())

Cantidad de valores nulos por columna:
listing_id    0
date          0
dtype: int64


In [51]:
#eliminar la columna 'adjusted_price' con 100% de valores NaN
df_calendar.drop(columns=['adjusted_price'], inplace=True)

df_calendar.to_csv('data_clean/calendar.csv', index=False)

In [54]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8885731 entries, 0 to 8885730
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64  
 1   date            object 
 2   available       object 
 3   price           object 
 4   minimum_nights  float64
 5   maximum_nights  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 406.8+ MB


In [55]:
# Calcular la moda de las columnas 'minimum_nights' y 'maximum_nights'
minimum_nights_mode = df_calendar['minimum_nights'].mode()[0]
maximum_nights_mode = df_calendar['maximum_nights'].mode()[0]

# Reemplazar valores nulos con la moda en cada columna
df_calendar['minimum_nights'].fillna(minimum_nights_mode, inplace=True)
df_calendar['maximum_nights'].fillna(maximum_nights_mode, inplace=True)

# Guardar el DataFrame modificado en el archivo especificado
df_calendar.to_csv('data_clean/calendar.csv', index=False)


In [61]:
#eliminar la columna 'neighbourhood_group' con 100% de valores NaN
df_neighbour.drop(columns=['neighbourhood_group'], inplace=True)

df_neighbour.to_csv('data_clean/neighbourhoods.csv', index=False)

In [5]:
#eliminar la columna 'adjusted_price' con 100% de valores NaN
df_calendar.drop(columns=['adjusted_price'], inplace=True)

#df_calendar.to_csv('data_clean/calendar.csv', index=False)

In [65]:
# Reemplazar los valores nulos de la columna 'comments' con 'Sin comentario' en df_calendar
if 'comments' in df_calendar.columns:
    df_calendar['comments'].fillna('Sin comentario', inplace=True)

df_calendar.to_csv('data_clean/reviews_detailed.csv', index=False)


In [69]:
# columna 'price' ajustar formato numérico, quitando símbolos como "$"
df_list_de['price'] = pd.to_numeric(df_list_de['price'].replace('[\$,]', '', regex=True), errors='coerce')

df_list_de.to_csv('data_clean/listings_detailed.csv', index=False)

In [82]:
# Convertir la columna 'date' en el dataset `df_calendar` a formato datetime
df_calendar['date'] = pd.to_datetime(df_calendar['date'], errors='coerce')
df_calendar.to_csv('data_clean/calendar.csv', index=False)

In [89]:
df_calendar['date']

0         2024-06-23
1         2024-06-24
2         2024-06-25
3         2024-06-26
4         2024-06-27
             ...    
8885726   2025-06-18
8885727   2025-06-19
8885728   2025-06-20
8885729   2025-06-21
8885730   2025-06-22
Name: date, Length: 8885731, dtype: datetime64[ns]

In [85]:
# Asegurarse de que 'price' esté en formato numérico, quitando símbolos como "$"
df_calendar['price'] = pd.to_numeric(df_calendar['price'].replace('[\$,]', '', regex=True), errors='coerce')

df_calendar.to_csv('data_clean/calendar.csv', index=False)

In [87]:
df_list_de.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24346 entries, 0 to 24345
Data columns (total 71 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            24346 non-null  int64  
 1   listing_url                                   24346 non-null  object 
 2   scrape_id                                     24346 non-null  int64  
 3   last_scraped                                  24346 non-null  object 
 4   source                                        24346 non-null  object 
 5   name                                          24346 non-null  object 
 6   description                                   24346 non-null  object 
 7   neighborhood_overview                         24346 non-null  object 
 8   picture_url                                   24346 non-null  object 
 9   host_id                                       24346 non-null 